In [1]:
import pandas as pd
import numpy as np
import math
import tensorflow as tf
from tfrecorder import TFrecorder
import struct
import os
import cv2 as cv
import subprocess
from subprocess import check_output
import scipy.io.wavfile as wav
PATH = '/USERS/d8182103/'
tfr = TFrecorder()
Model = tf.keras.models

In [2]:
def path_list(dataset='V2',setname='train',feature_type = 'mfcc'):
    audio_feature_path = '%s/firstimpression%s/%s/%s/' %(PATH, dataset, feature_type, setname)
    print(audio_feature_path)
    video_list = []
    path_list = []
    for folder in os.listdir(audio_feature_path):
        if '80_' in folder:
            for feature in os.listdir('%s/%s/' %(audio_feature_path,folder)):
                if '.htk' in feature:
                    video_list.append(feature.split('.htk')[0])
                    path_list.append('%s/%s/%s' %(audio_feature_path,folder,feature))
    path_df = pd.DataFrame({'video':video_list,'path':path_list},columns=['video','path'])
    return path_df.set_index('video')
def get_dataframe(df,dataset,setname):
    # read file containing the video name that will not be used
    delete_list = PATH+('firstimpression%s/tfrecord/text/%s/%s_videos_without_transcription.txt' %(dataset,setname,setname))
    if dataset =='V2':
        df = df.drop(columns=['text'])
    try:
        with open(delete_list,'r') as f:
            lines = f.readlines()
    except FileNotFoundError:
        print('firstimpression V1')
    else:
        # get indexes that should be removed from datafram
        remove_indexes = []
        for l in lines:
            remove_indexes.append(df[df['video']==l[:-1]].index[0])
        # drop rows of those indexes
        df = df.drop(np.array(remove_indexes))
    return df
def create_parser(data_info, retrieve_shape):

    names = data_info['name']
    types = data_info['type']
    shapes = data_info['shape']
    isbytes = data_info['isbyte']
    defaults = data_info['default']
    length_types = data_info['length_type']

    def parser(example_proto):
        def specify_features():
            specified_features = {}
            for i in np.arange(len(names)):
                # which type
                if isbytes[i]:
                    t = tf.string
                    s = ()
                else:
                    if types[i]=='uint8':
                        types[i]=tf.uint8
                    t = types[i]
                    s = shapes[i]
                # has default_value?
                if defaults[i] == np.NaN:
                    d = np.NaN
                else:
                    d = defaults[i]
                # length varies
                if length_types[i] =='fixed':
                    specified_features[names[i]] = tf.FixedLenFeature(s, t)
                elif length_types[i] =='var':
                    specified_features[names[i]] = tf.VarLenFeature(t)
                else:
                    raise TypeError("length_type is not one of 'var', 'fixed'")
            return specified_features


        # decode each parsed feature and reshape
        def decode_reshape():
            # store all decoded&shaped features
            final_features = {}
            for i in np.arange(len(names)):
                # exclude shape info
                if '_shape' not in names[i]:
                    # decode
                    if isbytes[i]:
                        # from byte format
                        decoded_value = tf.decode_raw(parsed_example[names[i]], types[i])
                        decoded_value = tf.cast(decoded_value, tf.float32)
                    else:
                        # Varlen value needs to be converted to dense format
                        if length_types[i] == 'var':
                            decoded_value = tf.sparse_tensor_to_dense(parsed_example[names[i]])
                        else:
                            decoded_value = parsed_example[names[i]]
                    # reshape
                    if '%s_shape' %names[i] in parsed_example.keys():
                        tf_shape = parsed_example['%s_shape' %names[i]]
                        decoded_value = tf.reshape(decoded_value, tf_shape)
                    final_features[names[i]] = decoded_value
                elif retrieve_shape:
                    final_features[names[i]] = parsed_example[names[i]]
            return final_features


        # create a dictionary to specify how to parse each feature 
        specified_features = specify_features()
        # parse all features of an example
        parsed_example = tf.parse_single_example(example_proto, specified_features)
        final_features = decode_reshape()

        return final_features
    return parser


def get_dataset(paths, data_info, retrieve_shape = False, num_parallel_calls=4, prefetch_buffer=2):

    filenames = paths
    
    data_info = pd.read_csv(data_info,dtype={'isbyte':bool})
    data_info['shape']=data_info['shape'].apply(lambda s: [int(i) for i in s[1:-1].split(',') if i !=''])

    #print(data_info)
    dataset = tf.data.TFRecordDataset(filenames)
    parse_function = create_parser(data_info, retrieve_shape)
    dataset = dataset.map(parse_function, num_parallel_calls=num_parallel_calls).prefetch(prefetch_buffer)
    return dataset

In [3]:
sess=tf.InteractiveSession()

In [4]:
def flowimages(ims):
    ims = ims.astype(np.uint8)
    hsv = np.zeros_like(ims[0])
    hsv[...,1] = 255
    def flow2(i1,i2):
        frame1 = cv.cvtColor(i1,cv.COLOR_BGR2GRAY)
        frame2 = cv.cvtColor(i2,cv.COLOR_BGR2GRAY)
        flow = cv.calcOpticalFlowFarneback(frame1,frame2, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        mag, ang = cv.cartToPolar(flow[...,0], flow[...,1])
        hsv[...,0] = ang*180/np.pi/2
        hsv[...,2] = cv.normalize(mag,None,0,255,cv.NORM_MINMAX)
        bgr = cv.cvtColor(hsv,cv.COLOR_HSV2BGR)
        return bgr
    num = ims.shape[0]
    tensors=[]
    for i in range(0,num-1):
        tensors.append(flow2(ims[i],ims[i+1]))
    return tensors

In [5]:
feature_name = 'opflow'
def save_audio_feature(dataset,setname):
    # read dataframe
    #df = pd.read_csv(PATH+'firstimpression%s/%s/text_and_labels.csv' %(dataset,setname))
    # get correct df
    #video_df = get_dataframe(df,dataset, setname)
    #path_df = path_wav_list(dataset=dataset,setname=setname)
    # read dataframe
    # model = Model.Model(inputs=InceptionV3.input, outputs=InceptionV3.get_layer('mixed8').output)
    df = pd.read_csv(PATH+'firstimpression%s/%s/text_and_labels.csv' %(dataset,setname))
    # get correct df
    video_df = get_dataframe(df,dataset, setname)

    num_examples = len(video_df)
    
    datapath = '/USERS/d8182103/'
    modalities = ['images']
    videoshape= [-1,224,224,3]
    videofeature='224x224'
    video_padding_info = {videofeature:videoshape}
    video_padding_value = {videofeature:0.0}
    video_info_path = '%s/firstimpression%s/tfrecord/images/%s.csv' %(datapath,dataset, videofeature)
    video_dataset_path = '%s/firstimpression%s/tfrecord/images/%s/%s.tfrecord' %(datapath, dataset, setname, videofeature)
    video_dataset = get_dataset(paths=video_dataset_path, data_info=video_info_path,  num_parallel_calls=1, prefetch_buffer=1)
    iterator = video_dataset.make_one_shot_iterator()
    example = iterator.get_next()
    
    data_info = pd.DataFrame({'name':[feature_name,feature_name+'_shape'],
                                 'type':['float32','int64'],
                                 'shape':[(2048,),(4,)],
                                 'isbyte':[True,False],
                                 "length_type":['fixed','fixed'],
                                 "default":[np.NaN,np.NaN]})

    data_info_path = PATH+'firstimpression%s/tfrecord/images/%s.csv' %(dataset,feature_name)
    data_info.to_csv(data_info_path,index=False)
    
    tfr = TFrecorder()
    writer = tf.python_io.TFRecordWriter(PATH+'firstimpression%s/tfrecord/images/%s/%s.tfrecord' %(dataset,setname,feature_name))
    #featuredata = np.load(PATH+'firstimpression%s/image_320x180/face_%s.npy' %(dataset,setname))
    #mean,std = np.load(PATH+'firstimpression%s/image_320x180/face_mean_std.npy' %(dataset))
    for i in np.arange(num_examples):
        features = {}
        
        #video = video_df.iloc[i]['video'].split('.mp4')[0]
        #wav_path = path_df.loc[video]['path']
        #mfcc_file_name = PATH+'firstimpression%s/mfcc_dir/%s/%s.mfc' %(dataset,setname,video) #path_df.loc[video]['path']
        inputs = example['224x224'].eval()
        out = np.array(flowimages(inputs))
        #audio_features = read_htk(mfcc_file_name)[0]
        #audio_features = fb_extractor(wav_path)
        #audio_features = np.array(audio_features,dtype='float32')
        #print(audio_features.shape)
        #sentence_based_norm = (audio_features-audio_features.mean())/audio_features.std()
        #sentence_based_norm = np.array(sentence_based_norm,dtype='float32')
        
        #featuredata[i] = (featuredata[i]-mean)/std
        #print(featuredata[i].mean(),featuredata[i].std(),featuredata[i].shape)
        print(out.shape)
        tfr.feature_writer(data_info.iloc[0], out.reshape(-1).astype('float32'), features)
        tfr.feature_writer(data_info.iloc[1], out.shape, features)
        
        tf_features = tf.train.Features(feature= features)
        tf_example = tf.train.Example(features = tf_features)
        tf_serialized = tf_example.SerializeToString()
        writer.write(tf_serialized)
        print(i)
        
    writer.close()

In [8]:
for dataset in ['V2']:
    for setname in ['train']:
        save_audio_feature(dataset,setname)
        print(dataset,setname)

(154, 224, 224, 3)
0
(154, 224, 224, 3)
1
(154, 224, 224, 3)
2
(154, 224, 224, 3)
3
(154, 224, 224, 3)
4
(154, 224, 224, 3)
5
(154, 224, 224, 3)
6
(154, 224, 224, 3)
7
(154, 224, 224, 3)
8
(154, 224, 224, 3)
9
(154, 224, 224, 3)
10
(154, 224, 224, 3)
11
(154, 224, 224, 3)
12
(154, 224, 224, 3)
13
(154, 224, 224, 3)
14
(154, 224, 224, 3)
15
(154, 224, 224, 3)
16
(154, 224, 224, 3)
17
(154, 224, 224, 3)
18
(154, 224, 224, 3)
19
(154, 224, 224, 3)
20
(154, 224, 224, 3)
21
(154, 224, 224, 3)
22
(154, 224, 224, 3)
23
(154, 224, 224, 3)
24
(154, 224, 224, 3)
25
(154, 224, 224, 3)
26
(154, 224, 224, 3)
27
(155, 224, 224, 3)
28
(154, 224, 224, 3)
29
(154, 224, 224, 3)
30
(154, 224, 224, 3)
31
(154, 224, 224, 3)
32
(154, 224, 224, 3)
33
(154, 224, 224, 3)
34
(154, 224, 224, 3)
35
(154, 224, 224, 3)
36
(154, 224, 224, 3)
37
(154, 224, 224, 3)
38
(154, 224, 224, 3)
39
(154, 224, 224, 3)
40
(154, 224, 224, 3)
41
(154, 224, 224, 3)
42
(154, 224, 224, 3)
43
(154, 224, 224, 3)
44
(154, 224, 224, 3)
4

(154, 224, 224, 3)
361
(154, 224, 224, 3)
362
(154, 224, 224, 3)
363
(154, 224, 224, 3)
364
(154, 224, 224, 3)
365
(154, 224, 224, 3)
366
(154, 224, 224, 3)
367
(154, 224, 224, 3)
368
(154, 224, 224, 3)
369
(154, 224, 224, 3)
370
(154, 224, 224, 3)
371
(154, 224, 224, 3)
372
(154, 224, 224, 3)
373
(154, 224, 224, 3)
374
(154, 224, 224, 3)
375
(154, 224, 224, 3)
376
(154, 224, 224, 3)
377
(154, 224, 224, 3)
378
(154, 224, 224, 3)
379
(154, 224, 224, 3)
380
(154, 224, 224, 3)
381
(154, 224, 224, 3)
382
(154, 224, 224, 3)
383
(154, 224, 224, 3)
384
(154, 224, 224, 3)
385
(154, 224, 224, 3)
386
(154, 224, 224, 3)
387
(154, 224, 224, 3)
388
(154, 224, 224, 3)
389
(154, 224, 224, 3)
390
(154, 224, 224, 3)
391
(154, 224, 224, 3)
392
(154, 224, 224, 3)
393
(154, 224, 224, 3)
394
(154, 224, 224, 3)
395
(154, 224, 224, 3)
396
(154, 224, 224, 3)
397
(154, 224, 224, 3)
398
(154, 224, 224, 3)
399
(154, 224, 224, 3)
400
(154, 224, 224, 3)
401
(154, 224, 224, 3)
402
(154, 224, 224, 3)
403
(154, 224, 

717
(154, 224, 224, 3)
718
(154, 224, 224, 3)
719
(154, 224, 224, 3)
720
(154, 224, 224, 3)
721
(154, 224, 224, 3)
722
(154, 224, 224, 3)
723
(154, 224, 224, 3)
724
(154, 224, 224, 3)
725
(154, 224, 224, 3)
726
(154, 224, 224, 3)
727
(154, 224, 224, 3)
728
(154, 224, 224, 3)
729
(154, 224, 224, 3)
730
(154, 224, 224, 3)
731
(154, 224, 224, 3)
732
(154, 224, 224, 3)
733
(154, 224, 224, 3)
734
(154, 224, 224, 3)
735
(154, 224, 224, 3)
736
(154, 224, 224, 3)
737
(154, 224, 224, 3)
738
(154, 224, 224, 3)
739
(154, 224, 224, 3)
740
(154, 224, 224, 3)
741
(154, 224, 224, 3)
742
(154, 224, 224, 3)
743
(154, 224, 224, 3)
744
(154, 224, 224, 3)
745
(154, 224, 224, 3)
746
(154, 224, 224, 3)
747
(154, 224, 224, 3)
748
(154, 224, 224, 3)
749
(154, 224, 224, 3)
750
(154, 224, 224, 3)
751
(154, 224, 224, 3)
752
(154, 224, 224, 3)
753
(154, 224, 224, 3)
754
(154, 224, 224, 3)
755
(154, 224, 224, 3)
756
(154, 224, 224, 3)
757
(154, 224, 224, 3)
758
(154, 224, 224, 3)
759
(154, 224, 224, 3)
760
(154, 2

(154, 224, 224, 3)
1071
(154, 224, 224, 3)
1072
(154, 224, 224, 3)
1073
(154, 224, 224, 3)
1074
(154, 224, 224, 3)
1075
(154, 224, 224, 3)
1076
(154, 224, 224, 3)
1077
(154, 224, 224, 3)
1078
(154, 224, 224, 3)
1079
(154, 224, 224, 3)
1080
(154, 224, 224, 3)
1081
(154, 224, 224, 3)
1082
(154, 224, 224, 3)
1083
(154, 224, 224, 3)
1084
(154, 224, 224, 3)
1085
(154, 224, 224, 3)
1086
(154, 224, 224, 3)
1087
(154, 224, 224, 3)
1088
(154, 224, 224, 3)
1089
(154, 224, 224, 3)
1090
(154, 224, 224, 3)
1091
(154, 224, 224, 3)
1092
(154, 224, 224, 3)
1093
(154, 224, 224, 3)
1094
(154, 224, 224, 3)
1095
(154, 224, 224, 3)
1096
(154, 224, 224, 3)
1097
(154, 224, 224, 3)
1098
(154, 224, 224, 3)
1099
(154, 224, 224, 3)
1100
(154, 224, 224, 3)
1101
(154, 224, 224, 3)
1102
(154, 224, 224, 3)
1103
(154, 224, 224, 3)
1104
(154, 224, 224, 3)
1105
(154, 224, 224, 3)
1106
(154, 224, 224, 3)
1107
(154, 224, 224, 3)
1108
(154, 224, 224, 3)
1109
(154, 224, 224, 3)
1110
(154, 224, 224, 3)
1111
(154, 224, 224, 

1412
(154, 224, 224, 3)
1413
(154, 224, 224, 3)
1414
(154, 224, 224, 3)
1415
(154, 224, 224, 3)
1416
(154, 224, 224, 3)
1417
(154, 224, 224, 3)
1418
(154, 224, 224, 3)
1419
(154, 224, 224, 3)
1420
(154, 224, 224, 3)
1421
(154, 224, 224, 3)
1422
(154, 224, 224, 3)
1423
(154, 224, 224, 3)
1424
(154, 224, 224, 3)
1425
(154, 224, 224, 3)
1426
(154, 224, 224, 3)
1427
(154, 224, 224, 3)
1428
(154, 224, 224, 3)
1429
(154, 224, 224, 3)
1430
(154, 224, 224, 3)
1431
(154, 224, 224, 3)
1432
(154, 224, 224, 3)
1433
(154, 224, 224, 3)
1434
(154, 224, 224, 3)
1435
(154, 224, 224, 3)
1436
(154, 224, 224, 3)
1437
(154, 224, 224, 3)
1438
(154, 224, 224, 3)
1439
(154, 224, 224, 3)
1440
(154, 224, 224, 3)
1441
(154, 224, 224, 3)
1442
(154, 224, 224, 3)
1443
(154, 224, 224, 3)
1444
(154, 224, 224, 3)
1445
(154, 224, 224, 3)
1446
(154, 224, 224, 3)
1447
(154, 224, 224, 3)
1448
(154, 224, 224, 3)
1449
(154, 224, 224, 3)
1450
(154, 224, 224, 3)
1451
(155, 224, 224, 3)
1452
(154, 224, 224, 3)
1453
(154, 224, 

...

4490
(154, 224, 224, 3)
4491
(154, 224, 224, 3)
4492
(154, 224, 224, 3)
4493
(154, 224, 224, 3)
4494
(154, 224, 224, 3)
4495
(154, 224, 224, 3)
4496
(154, 224, 224, 3)
4497
(154, 224, 224, 3)
4498
(154, 224, 224, 3)
4499
(154, 224, 224, 3)
4500
(154, 224, 224, 3)
4501
(154, 224, 224, 3)
4502
(154, 224, 224, 3)
4503
(154, 224, 224, 3)
4504
(154, 224, 224, 3)
4505
(154, 224, 224, 3)
4506
(154, 224, 224, 3)
4507
(154, 224, 224, 3)
4508
(154, 224, 224, 3)
4509
(154, 224, 224, 3)
4510
(154, 224, 224, 3)
4511
(154, 224, 224, 3)
4512
(154, 224, 224, 3)
4513
(154, 224, 224, 3)
4514
(154, 224, 224, 3)
4515
(154, 224, 224, 3)
4516
(154, 224, 224, 3)
4517
(154, 224, 224, 3)
4518
(154, 224, 224, 3)
4519
(154, 224, 224, 3)
4520
(154, 224, 224, 3)
4521
(154, 224, 224, 3)
4522
(154, 224, 224, 3)
4523
(154, 224, 224, 3)
4524
(154, 224, 224, 3)
4525
(154, 224, 224, 3)
4526
(154, 224, 224, 3)
4527
(154, 224, 224, 3)
4528
(154, 224, 224, 3)
4529
(154, 224, 224, 3)
4530
(154, 224, 224, 3)
4531
(154, 224, 

4832
(154, 224, 224, 3)
4833
(154, 224, 224, 3)
4834
(154, 224, 224, 3)
4835
(154, 224, 224, 3)
4836
(154, 224, 224, 3)
4837
(154, 224, 224, 3)
4838
(154, 224, 224, 3)
4839
(154, 224, 224, 3)
4840
(154, 224, 224, 3)
4841
(154, 224, 224, 3)
4842
(154, 224, 224, 3)
4843
(154, 224, 224, 3)
4844
(154, 224, 224, 3)
4845
(154, 224, 224, 3)
4846
(154, 224, 224, 3)
4847
(154, 224, 224, 3)
4848
(154, 224, 224, 3)
4849
(154, 224, 224, 3)
4850
(154, 224, 224, 3)
4851
(155, 224, 224, 3)
4852
(154, 224, 224, 3)
4853
(154, 224, 224, 3)
4854
(154, 224, 224, 3)
4855
(154, 224, 224, 3)
4856
(154, 224, 224, 3)
4857
(154, 224, 224, 3)
4858
(154, 224, 224, 3)
4859
(154, 224, 224, 3)
4860
(154, 224, 224, 3)
4861
(154, 224, 224, 3)
4862
(154, 224, 224, 3)
4863
(154, 224, 224, 3)
4864
(154, 224, 224, 3)
4865
(154, 224, 224, 3)
4866
(154, 224, 224, 3)
4867
(154, 224, 224, 3)
4868
(154, 224, 224, 3)
4869
(154, 224, 224, 3)
4870
(154, 224, 224, 3)
4871
(154, 224, 224, 3)
4872
(154, 224, 224, 3)
4873
(154, 224, 

5174
(154, 224, 224, 3)
5175
(154, 224, 224, 3)
5176
(154, 224, 224, 3)
5177
(154, 224, 224, 3)
5178
(154, 224, 224, 3)
5179
(154, 224, 224, 3)
5180
(154, 224, 224, 3)
5181
(154, 224, 224, 3)
5182
(152, 224, 224, 3)
5183
(154, 224, 224, 3)
5184
(154, 224, 224, 3)
5185
(154, 224, 224, 3)
5186
(154, 224, 224, 3)
5187
(154, 224, 224, 3)
5188
(154, 224, 224, 3)
5189
(154, 224, 224, 3)
5190
(154, 224, 224, 3)
5191
(154, 224, 224, 3)
5192
(154, 224, 224, 3)
5193
(154, 224, 224, 3)
5194
(154, 224, 224, 3)
5195
(154, 224, 224, 3)
5196
(154, 224, 224, 3)
5197
(154, 224, 224, 3)
5198
(154, 224, 224, 3)
5199
(154, 224, 224, 3)
5200
(154, 224, 224, 3)
5201
(154, 224, 224, 3)
5202
(154, 224, 224, 3)
5203
(154, 224, 224, 3)
5204
(154, 224, 224, 3)
5205
(154, 224, 224, 3)
5206
(154, 224, 224, 3)
5207
(154, 224, 224, 3)
5208
(154, 224, 224, 3)
5209
(154, 224, 224, 3)
5210
(154, 224, 224, 3)
5211
(154, 224, 224, 3)
5212
(154, 224, 224, 3)
5213
(154, 224, 224, 3)
5214
(154, 224, 224, 3)
5215
(154, 224, 

5516
(154, 224, 224, 3)
5517
(154, 224, 224, 3)
5518
(154, 224, 224, 3)
5519
(154, 224, 224, 3)
5520
(154, 224, 224, 3)
5521
(154, 224, 224, 3)
5522
(154, 224, 224, 3)
5523
(154, 224, 224, 3)
5524
(154, 224, 224, 3)
5525
(154, 224, 224, 3)
5526
(154, 224, 224, 3)
5527
(154, 224, 224, 3)
5528
(154, 224, 224, 3)
5529
(154, 224, 224, 3)
5530
(154, 224, 224, 3)
5531
(154, 224, 224, 3)
5532
(154, 224, 224, 3)
5533
(154, 224, 224, 3)
5534
(154, 224, 224, 3)
5535
(154, 224, 224, 3)
5536
(154, 224, 224, 3)
5537
(154, 224, 224, 3)
5538
(154, 224, 224, 3)
5539
(154, 224, 224, 3)
5540
(154, 224, 224, 3)
5541
(154, 224, 224, 3)
5542
(154, 224, 224, 3)
5543
(154, 224, 224, 3)
5544
(154, 224, 224, 3)
5545
(154, 224, 224, 3)
5546
(154, 224, 224, 3)
5547
(154, 224, 224, 3)
5548
(155, 224, 224, 3)
5549
(154, 224, 224, 3)
5550
(154, 224, 224, 3)
5551
(154, 224, 224, 3)
5552
(154, 224, 224, 3)
5553
(154, 224, 224, 3)
5554
(154, 224, 224, 3)
5555
(154, 224, 224, 3)
5556
(154, 224, 224, 3)
5557
(154, 224, 

5858
(154, 224, 224, 3)
5859
(154, 224, 224, 3)
5860
(154, 224, 224, 3)
5861
(154, 224, 224, 3)
5862
(154, 224, 224, 3)
5863
(154, 224, 224, 3)
5864
(154, 224, 224, 3)
5865
(154, 224, 224, 3)
5866
(154, 224, 224, 3)
5867
(154, 224, 224, 3)
5868
(154, 224, 224, 3)
5869
(154, 224, 224, 3)
5870
(154, 224, 224, 3)
5871
(154, 224, 224, 3)
5872
(154, 224, 224, 3)
5873
(154, 224, 224, 3)
5874
(154, 224, 224, 3)
5875
(154, 224, 224, 3)
5876
(154, 224, 224, 3)
5877
(154, 224, 224, 3)
5878
(154, 224, 224, 3)
5879
(154, 224, 224, 3)
5880
(154, 224, 224, 3)
5881
(154, 224, 224, 3)
5882
(154, 224, 224, 3)
5883
(142, 224, 224, 3)
5884
(154, 224, 224, 3)
5885
(154, 224, 224, 3)
5886
(154, 224, 224, 3)
5887
(154, 224, 224, 3)
5888
(154, 224, 224, 3)
5889
(154, 224, 224, 3)
5890
(154, 224, 224, 3)
5891
(154, 224, 224, 3)
5892
(154, 224, 224, 3)
5893
(154, 224, 224, 3)
5894
(154, 224, 224, 3)
5895
(154, 224, 224, 3)
5896
(154, 224, 224, 3)
5897
(154, 224, 224, 3)
5898
(154, 224, 224, 3)
5899
(154, 224, 

In [6]:
for dataset in ['V2']:
    for setname in ['vali','test']:
        save_audio_feature(dataset,setname)
        print(dataset,setname)

(154, 224, 224, 3)
0
(154, 224, 224, 3)
1
(154, 224, 224, 3)
2
(154, 224, 224, 3)
3
(154, 224, 224, 3)
4
(154, 224, 224, 3)
5
(154, 224, 224, 3)
6
(154, 224, 224, 3)
7
(154, 224, 224, 3)
8
(154, 224, 224, 3)
9
(154, 224, 224, 3)
10
(154, 224, 224, 3)
11
(154, 224, 224, 3)
12
(154, 224, 224, 3)
13
(154, 224, 224, 3)
14
(154, 224, 224, 3)
15
(154, 224, 224, 3)
16
(154, 224, 224, 3)
17
(154, 224, 224, 3)
18
(154, 224, 224, 3)
19
(154, 224, 224, 3)
20
(154, 224, 224, 3)
21
(154, 224, 224, 3)
22
(154, 224, 224, 3)
23
(154, 224, 224, 3)
24
(154, 224, 224, 3)
25
(154, 224, 224, 3)
26
(154, 224, 224, 3)
27
(154, 224, 224, 3)
28
(154, 224, 224, 3)
29
(154, 224, 224, 3)
30
(154, 224, 224, 3)
31
(154, 224, 224, 3)
32
(154, 224, 224, 3)
33
(154, 224, 224, 3)
34
(154, 224, 224, 3)
35
(154, 224, 224, 3)
36
(154, 224, 224, 3)
37
(154, 224, 224, 3)
38
(154, 224, 224, 3)
39
(154, 224, 224, 3)
40
(154, 224, 224, 3)
41
(154, 224, 224, 3)
42
(154, 224, 224, 3)
43
(154, 224, 224, 3)
44
(154, 224, 224, 3)
4

(154, 224, 224, 3)
361
(154, 224, 224, 3)
362
(154, 224, 224, 3)
363
(154, 224, 224, 3)
364
(154, 224, 224, 3)
365
(154, 224, 224, 3)
366
(154, 224, 224, 3)
367
(154, 224, 224, 3)
368
(154, 224, 224, 3)
369
(154, 224, 224, 3)
370
(154, 224, 224, 3)
371
(154, 224, 224, 3)
372
(154, 224, 224, 3)
373
(154, 224, 224, 3)
374
(154, 224, 224, 3)
375
(154, 224, 224, 3)
376
(154, 224, 224, 3)
377
(154, 224, 224, 3)
378
(154, 224, 224, 3)
379
(154, 224, 224, 3)
380
(154, 224, 224, 3)
381
(154, 224, 224, 3)
382
(154, 224, 224, 3)
383
(154, 224, 224, 3)
384
(154, 224, 224, 3)
385
(154, 224, 224, 3)
386
(154, 224, 224, 3)
387
(154, 224, 224, 3)
388
(154, 224, 224, 3)
389
(154, 224, 224, 3)
390
(154, 224, 224, 3)
391
(154, 224, 224, 3)
392
(154, 224, 224, 3)
393
(154, 224, 224, 3)
394
(154, 224, 224, 3)
395
(154, 224, 224, 3)
396
(154, 224, 224, 3)
397
(154, 224, 224, 3)
398
(154, 224, 224, 3)
399
(154, 224, 224, 3)
400
(154, 224, 224, 3)
401
(154, 224, 224, 3)
402
(154, 224, 224, 3)
403
(154, 224, 

717
(154, 224, 224, 3)
718
(154, 224, 224, 3)
719
(154, 224, 224, 3)
720
(154, 224, 224, 3)
721
(154, 224, 224, 3)
722
(154, 224, 224, 3)
723
(154, 224, 224, 3)
724
(154, 224, 224, 3)
725
(154, 224, 224, 3)
726
(154, 224, 224, 3)
727
(154, 224, 224, 3)
728
(154, 224, 224, 3)
729
(154, 224, 224, 3)
730
(154, 224, 224, 3)
731
(154, 224, 224, 3)
732
(154, 224, 224, 3)
733
(154, 224, 224, 3)
734
(154, 224, 224, 3)
735
(154, 224, 224, 3)
736
(154, 224, 224, 3)
737
(154, 224, 224, 3)
738
(154, 224, 224, 3)
739
(154, 224, 224, 3)
740
(154, 224, 224, 3)
741
(154, 224, 224, 3)
742
(154, 224, 224, 3)
743
(154, 224, 224, 3)
744
(154, 224, 224, 3)
745
(154, 224, 224, 3)
746
(154, 224, 224, 3)
747
(154, 224, 224, 3)
748
(154, 224, 224, 3)
749
(154, 224, 224, 3)
750
(154, 224, 224, 3)
751
(154, 224, 224, 3)
752
(154, 224, 224, 3)
753
(154, 224, 224, 3)
754
(154, 224, 224, 3)
755
(154, 224, 224, 3)
756
(154, 224, 224, 3)
757
(154, 224, 224, 3)
758
(154, 224, 224, 3)
759
(154, 224, 224, 3)
760
(154, 2

(154, 224, 224, 3)
1071
(154, 224, 224, 3)
1072
(154, 224, 224, 3)
1073
(154, 224, 224, 3)
1074
(154, 224, 224, 3)
1075
(154, 224, 224, 3)
1076
(154, 224, 224, 3)
1077
(154, 224, 224, 3)
1078
(155, 224, 224, 3)
1079
(154, 224, 224, 3)
1080
(154, 224, 224, 3)
1081
(154, 224, 224, 3)
1082
(154, 224, 224, 3)
1083
(154, 224, 224, 3)
1084
(154, 224, 224, 3)
1085
(154, 224, 224, 3)
1086
(154, 224, 224, 3)
1087
(154, 224, 224, 3)
1088
(154, 224, 224, 3)
1089
(154, 224, 224, 3)
1090
(154, 224, 224, 3)
1091
(154, 224, 224, 3)
1092
(154, 224, 224, 3)
1093
(154, 224, 224, 3)
1094
(154, 224, 224, 3)
1095
(154, 224, 224, 3)
1096
(154, 224, 224, 3)
1097
(154, 224, 224, 3)
1098
(154, 224, 224, 3)
1099
(154, 224, 224, 3)
1100
(154, 224, 224, 3)
1101
(154, 224, 224, 3)
1102
(154, 224, 224, 3)
1103
(154, 224, 224, 3)
1104
(154, 224, 224, 3)
1105
(154, 224, 224, 3)
1106
(154, 224, 224, 3)
1107
(154, 224, 224, 3)
1108
(154, 224, 224, 3)
1109
(154, 224, 224, 3)
1110
(154, 224, 224, 3)
1111
(154, 224, 224, 

1412
(154, 224, 224, 3)
1413
(154, 224, 224, 3)
1414
(154, 224, 224, 3)
1415
(154, 224, 224, 3)
1416
(154, 224, 224, 3)
1417
(154, 224, 224, 3)
1418
(154, 224, 224, 3)
1419
(154, 224, 224, 3)
1420
(154, 224, 224, 3)
1421
(154, 224, 224, 3)
1422
(154, 224, 224, 3)
1423
(154, 224, 224, 3)
1424
(154, 224, 224, 3)
1425
(154, 224, 224, 3)
1426
(154, 224, 224, 3)
1427
(154, 224, 224, 3)
1428
(154, 224, 224, 3)
1429
(154, 224, 224, 3)
1430
(154, 224, 224, 3)
1431
(154, 224, 224, 3)
1432
(154, 224, 224, 3)
1433
(155, 224, 224, 3)
1434
(154, 224, 224, 3)
1435
(154, 224, 224, 3)
1436
(154, 224, 224, 3)
1437
(154, 224, 224, 3)
1438
(154, 224, 224, 3)
1439
(154, 224, 224, 3)
1440
(154, 224, 224, 3)
1441
(154, 224, 224, 3)
1442
(154, 224, 224, 3)
1443
(154, 224, 224, 3)
1444
(154, 224, 224, 3)
1445
(154, 224, 224, 3)
1446
(154, 224, 224, 3)
1447
(154, 224, 224, 3)
1448
(107, 224, 224, 3)
1449
(154, 224, 224, 3)
1450
(154, 224, 224, 3)
1451
(154, 224, 224, 3)
1452
(154, 224, 224, 3)
1453
(154, 224, 

...

461
(154, 224, 224, 3)
462
(154, 224, 224, 3)
463
(154, 224, 224, 3)
464
(154, 224, 224, 3)
465
(154, 224, 224, 3)
466
(154, 224, 224, 3)
467
(154, 224, 224, 3)
468
(154, 224, 224, 3)
469
(154, 224, 224, 3)
470
(154, 224, 224, 3)
471
(154, 224, 224, 3)
472
(154, 224, 224, 3)
473
(154, 224, 224, 3)
474
(154, 224, 224, 3)
475
(154, 224, 224, 3)
476
(154, 224, 224, 3)
477
(154, 224, 224, 3)
478
(154, 224, 224, 3)
479
(154, 224, 224, 3)
480
(154, 224, 224, 3)
481
(154, 224, 224, 3)
482
(154, 224, 224, 3)
483
(154, 224, 224, 3)
484
(154, 224, 224, 3)
485
(154, 224, 224, 3)
486
(154, 224, 224, 3)
487
(154, 224, 224, 3)
488
(154, 224, 224, 3)
489
(154, 224, 224, 3)
490
(154, 224, 224, 3)
491
(154, 224, 224, 3)
492
(154, 224, 224, 3)
493
(154, 224, 224, 3)
494
(154, 224, 224, 3)
495
(154, 224, 224, 3)
496
(154, 224, 224, 3)
497
(154, 224, 224, 3)
498
(154, 224, 224, 3)
499
(154, 224, 224, 3)
500
(154, 224, 224, 3)
501
(154, 224, 224, 3)
502
(154, 224, 224, 3)
503
(154, 224, 224, 3)
504
(154, 2

818
(154, 224, 224, 3)
819
(154, 224, 224, 3)
820
(154, 224, 224, 3)
821
(154, 224, 224, 3)
822
(154, 224, 224, 3)
823
(154, 224, 224, 3)
824
(154, 224, 224, 3)
825
(154, 224, 224, 3)
826
(154, 224, 224, 3)
827
(154, 224, 224, 3)
828
(154, 224, 224, 3)
829
(154, 224, 224, 3)
830
(154, 224, 224, 3)
831
(154, 224, 224, 3)
832
(154, 224, 224, 3)
833
(154, 224, 224, 3)
834
(154, 224, 224, 3)
835
(154, 224, 224, 3)
836
(154, 224, 224, 3)
837
(154, 224, 224, 3)
838
(154, 224, 224, 3)
839
(154, 224, 224, 3)
840
(154, 224, 224, 3)
841
(154, 224, 224, 3)
842
(154, 224, 224, 3)
843
(154, 224, 224, 3)
844
(154, 224, 224, 3)
845
(154, 224, 224, 3)
846
(154, 224, 224, 3)
847
(154, 224, 224, 3)
848
(154, 224, 224, 3)
849
(154, 224, 224, 3)
850
(154, 224, 224, 3)
851
(154, 224, 224, 3)
852
(154, 224, 224, 3)
853
(154, 224, 224, 3)
854
(154, 224, 224, 3)
855
(154, 224, 224, 3)
856
(154, 224, 224, 3)
857
(154, 224, 224, 3)
858
(154, 224, 224, 3)
859
(154, 224, 224, 3)
860
(154, 224, 224, 3)
861
(154, 2

1167
(154, 224, 224, 3)
1168
(154, 224, 224, 3)
1169
(154, 224, 224, 3)
1170
(154, 224, 224, 3)
1171
(154, 224, 224, 3)
1172
(154, 224, 224, 3)
1173
(154, 224, 224, 3)
1174
(154, 224, 224, 3)
1175
(154, 224, 224, 3)
1176
(154, 224, 224, 3)
1177
(154, 224, 224, 3)
1178
(154, 224, 224, 3)
1179
(154, 224, 224, 3)
1180
(154, 224, 224, 3)
1181
(154, 224, 224, 3)
1182
(154, 224, 224, 3)
1183
(154, 224, 224, 3)
1184
(154, 224, 224, 3)
1185
(154, 224, 224, 3)
1186
(154, 224, 224, 3)
1187
(154, 224, 224, 3)
1188
(154, 224, 224, 3)
1189
(154, 224, 224, 3)
1190
(154, 224, 224, 3)
1191
(154, 224, 224, 3)
1192
(154, 224, 224, 3)
1193
(154, 224, 224, 3)
1194
(154, 224, 224, 3)
1195
(154, 224, 224, 3)
1196
(154, 224, 224, 3)
1197
(154, 224, 224, 3)
1198
(154, 224, 224, 3)
1199
(154, 224, 224, 3)
1200
(154, 224, 224, 3)
1201
(154, 224, 224, 3)
1202
(154, 224, 224, 3)
1203
(154, 224, 224, 3)
1204
(154, 224, 224, 3)
1205
(154, 224, 224, 3)
1206
(154, 224, 224, 3)
1207
(154, 224, 224, 3)
1208
(96, 224, 2

1509
(154, 224, 224, 3)
1510
(154, 224, 224, 3)
1511
(154, 224, 224, 3)
1512
(154, 224, 224, 3)
1513
(154, 224, 224, 3)
1514
(155, 224, 224, 3)
1515
(154, 224, 224, 3)
1516
(154, 224, 224, 3)
1517
(148, 224, 224, 3)
1518
(154, 224, 224, 3)
1519
(154, 224, 224, 3)
1520
(154, 224, 224, 3)
1521
(154, 224, 224, 3)
1522
(154, 224, 224, 3)
1523
(154, 224, 224, 3)
1524
(154, 224, 224, 3)
1525
(154, 224, 224, 3)
1526
(154, 224, 224, 3)
1527
(154, 224, 224, 3)
1528
(154, 224, 224, 3)
1529
(154, 224, 224, 3)
1530
(154, 224, 224, 3)
1531
(154, 224, 224, 3)
1532
(154, 224, 224, 3)
1533
(154, 224, 224, 3)
1534
(154, 224, 224, 3)
1535
(154, 224, 224, 3)
1536
(154, 224, 224, 3)
1537
(154, 224, 224, 3)
1538
(154, 224, 224, 3)
1539
(154, 224, 224, 3)
1540
(154, 224, 224, 3)
1541
(154, 224, 224, 3)
1542
(154, 224, 224, 3)
1543
(154, 224, 224, 3)
1544
(154, 224, 224, 3)
1545
(154, 224, 224, 3)
1546
(154, 224, 224, 3)
1547
(154, 224, 224, 3)
1548
(154, 224, 224, 3)
1549
(154, 224, 224, 3)
1550
(154, 224, 

1851
(154, 224, 224, 3)
1852
(154, 224, 224, 3)
1853
(154, 224, 224, 3)
1854
(154, 224, 224, 3)
1855
(154, 224, 224, 3)
1856
(154, 224, 224, 3)
1857
(154, 224, 224, 3)
1858
(154, 224, 224, 3)
1859
(154, 224, 224, 3)
1860
(154, 224, 224, 3)
1861
(154, 224, 224, 3)
1862
(154, 224, 224, 3)
1863
(154, 224, 224, 3)
1864
(154, 224, 224, 3)
1865
(154, 224, 224, 3)
1866
(154, 224, 224, 3)
1867
(154, 224, 224, 3)
1868
(154, 224, 224, 3)
1869
(154, 224, 224, 3)
1870
(154, 224, 224, 3)
1871
(154, 224, 224, 3)
1872
(154, 224, 224, 3)
1873
(154, 224, 224, 3)
1874
(154, 224, 224, 3)
1875
(154, 224, 224, 3)
1876
(154, 224, 224, 3)
1877
(154, 224, 224, 3)
1878
(154, 224, 224, 3)
1879
(155, 224, 224, 3)
1880
(154, 224, 224, 3)
1881
(154, 224, 224, 3)
1882
(154, 224, 224, 3)
1883
(154, 224, 224, 3)
1884
(154, 224, 224, 3)
1885
(154, 224, 224, 3)
1886
(154, 224, 224, 3)
1887
(154, 224, 224, 3)
1888
(154, 224, 224, 3)
1889
(154, 224, 224, 3)
1890
(154, 224, 224, 3)
1891
(154, 224, 224, 3)
1892
(154, 224, 

In [ ]:
for dataset in ['V1']:
    for setname in ['train','vali']:
        save_audio_feature(dataset,setname)
        print(dataset,setname)

firstimpression V1
(155, 4, 8, 1280)
0
(155, 4, 8, 1280)
1
(155, 4, 8, 1280)
2
(155, 4, 8, 1280)
3
(155, 4, 8, 1280)
4
(155, 4, 8, 1280)
5
(155, 4, 8, 1280)
6
(155, 4, 8, 1280)
7
(155, 4, 8, 1280)
8
(155, 4, 8, 1280)
9
(155, 4, 8, 1280)
10
(155, 4, 8, 1280)
11
(155, 4, 8, 1280)
12
(155, 4, 8, 1280)
13
(155, 4, 8, 1280)
14
(155, 4, 8, 1280)
15
(155, 4, 8, 1280)
16
(155, 4, 8, 1280)
17
(155, 4, 8, 1280)
18
(155, 4, 8, 1280)
19
(155, 4, 8, 1280)
20
(155, 4, 8, 1280)
21
(155, 4, 8, 1280)
22
(155, 4, 8, 1280)
23
(155, 4, 8, 1280)
24
(155, 4, 8, 1280)
25
(155, 4, 8, 1280)
26
(155, 4, 8, 1280)
27
(156, 4, 8, 1280)
28
(155, 4, 8, 1280)
29
(155, 4, 8, 1280)
30
(155, 4, 8, 1280)
31
(155, 4, 8, 1280)
32
(155, 4, 8, 1280)
33
(155, 4, 8, 1280)
34
(155, 4, 8, 1280)
35
(155, 4, 8, 1280)
36
(155, 4, 8, 1280)
37
(155, 4, 8, 1280)
38
(155, 4, 8, 1280)
39
(155, 4, 8, 1280)
40
(155, 4, 8, 1280)
41
(155, 4, 8, 1280)
42
(155, 4, 8, 1280)
43
(155, 4, 8, 1280)
44
(155, 4, 8, 1280)
45
(155, 4, 8, 1280)
46
(155

(155, 4, 8, 1280)
377
(155, 4, 8, 1280)
378
(155, 4, 8, 1280)
379
(155, 4, 8, 1280)
380
(155, 4, 8, 1280)
381
(155, 4, 8, 1280)
382
(155, 4, 8, 1280)
383
(155, 4, 8, 1280)
384
(155, 4, 8, 1280)
385
(155, 4, 8, 1280)
386
(155, 4, 8, 1280)
387
(155, 4, 8, 1280)
388
(155, 4, 8, 1280)
389
(155, 4, 8, 1280)
390
(155, 4, 8, 1280)
391
(155, 4, 8, 1280)
392
(155, 4, 8, 1280)
393
(155, 4, 8, 1280)
394
(155, 4, 8, 1280)
395
(155, 4, 8, 1280)
396
(155, 4, 8, 1280)
397
(155, 4, 8, 1280)
398
(155, 4, 8, 1280)
399
(155, 4, 8, 1280)
400
(155, 4, 8, 1280)
401
(155, 4, 8, 1280)
402
(155, 4, 8, 1280)
403
(155, 4, 8, 1280)
404
(155, 4, 8, 1280)
405
(155, 4, 8, 1280)
406
(155, 4, 8, 1280)
407
(155, 4, 8, 1280)
408
(155, 4, 8, 1280)
409
(155, 4, 8, 1280)
410
(155, 4, 8, 1280)
411
(155, 4, 8, 1280)
412
(155, 4, 8, 1280)
413
(155, 4, 8, 1280)
414
(155, 4, 8, 1280)
415
(155, 4, 8, 1280)
416
(155, 4, 8, 1280)
417
(155, 4, 8, 1280)
418
(155, 4, 8, 1280)
419
(155, 4, 8, 1280)
420
(155, 4, 8, 1280)
421
(155, 4, 8

(155, 4, 8, 1280)
750
(155, 4, 8, 1280)
751
(155, 4, 8, 1280)
752
(155, 4, 8, 1280)
753
(155, 4, 8, 1280)
754
(155, 4, 8, 1280)
755
(155, 4, 8, 1280)
756
(155, 4, 8, 1280)
757
(155, 4, 8, 1280)
758
(155, 4, 8, 1280)
759
(155, 4, 8, 1280)
760
(155, 4, 8, 1280)
761
(155, 4, 8, 1280)
762
(155, 4, 8, 1280)
763
(155, 4, 8, 1280)
764
(155, 4, 8, 1280)
765
(155, 4, 8, 1280)
766
(155, 4, 8, 1280)
767
(155, 4, 8, 1280)
768
(155, 4, 8, 1280)
769
(155, 4, 8, 1280)
770
(155, 4, 8, 1280)
771
(155, 4, 8, 1280)
772
(155, 4, 8, 1280)
773
(155, 4, 8, 1280)
774
(155, 4, 8, 1280)
775
(155, 4, 8, 1280)
776
(155, 4, 8, 1280)
777
(155, 4, 8, 1280)
778
(155, 4, 8, 1280)
779
(155, 4, 8, 1280)
780
(155, 4, 8, 1280)
781
(156, 4, 8, 1280)
782
(155, 4, 8, 1280)
783
(155, 4, 8, 1280)
784
(155, 4, 8, 1280)
785
(155, 4, 8, 1280)
786
(155, 4, 8, 1280)
787
(155, 4, 8, 1280)
788
(155, 4, 8, 1280)
789
(155, 4, 8, 1280)
790
(155, 4, 8, 1280)
791
(155, 4, 8, 1280)
792
(155, 4, 8, 1280)
793
(155, 4, 8, 1280)
794
(155, 4, 8

(155, 4, 8, 1280)
1118
(155, 4, 8, 1280)
1119
(155, 4, 8, 1280)
1120
(155, 4, 8, 1280)
1121
(155, 4, 8, 1280)
1122
(155, 4, 8, 1280)
1123
(155, 4, 8, 1280)
1124
(155, 4, 8, 1280)
1125
(155, 4, 8, 1280)
1126
(155, 4, 8, 1280)
1127
(155, 4, 8, 1280)
1128
(155, 4, 8, 1280)
1129
(155, 4, 8, 1280)
1130
(155, 4, 8, 1280)
1131
(155, 4, 8, 1280)
1132
(155, 4, 8, 1280)
1133
(155, 4, 8, 1280)
1134
(155, 4, 8, 1280)
1135
(155, 4, 8, 1280)
1136
(155, 4, 8, 1280)
1137
(155, 4, 8, 1280)
1138
(155, 4, 8, 1280)
1139
(155, 4, 8, 1280)
1140
(155, 4, 8, 1280)
1141
(155, 4, 8, 1280)
1142
(155, 4, 8, 1280)
1143
(155, 4, 8, 1280)
1144
(155, 4, 8, 1280)
1145
(155, 4, 8, 1280)
1146
(155, 4, 8, 1280)
1147
(156, 4, 8, 1280)
1148
(155, 4, 8, 1280)
1149
(155, 4, 8, 1280)
1150
(155, 4, 8, 1280)
1151
(155, 4, 8, 1280)
1152
(155, 4, 8, 1280)
1153
(155, 4, 8, 1280)
1154
(155, 4, 8, 1280)
1155
(155, 4, 8, 1280)
1156
(155, 4, 8, 1280)
1157
(155, 4, 8, 1280)
1158
(155, 4, 8, 1280)
1159
(155, 4, 8, 1280)
1160
(155, 4, 8,

(155, 4, 8, 1280)
1475
(155, 4, 8, 1280)
1476
(155, 4, 8, 1280)
1477
(156, 4, 8, 1280)
1478
(155, 4, 8, 1280)
1479
(155, 4, 8, 1280)
1480
(155, 4, 8, 1280)
1481
(155, 4, 8, 1280)
1482
(155, 4, 8, 1280)
1483
(155, 4, 8, 1280)
1484
(155, 4, 8, 1280)
1485
(155, 4, 8, 1280)
1486
(155, 4, 8, 1280)
1487
(155, 4, 8, 1280)
1488
(155, 4, 8, 1280)
1489
(155, 4, 8, 1280)
1490
(155, 4, 8, 1280)
1491
(155, 4, 8, 1280)
1492
(155, 4, 8, 1280)
1493
(155, 4, 8, 1280)
1494
(155, 4, 8, 1280)
1495
(155, 4, 8, 1280)
1496
(155, 4, 8, 1280)
1497
(155, 4, 8, 1280)
1498
(155, 4, 8, 1280)
1499
(155, 4, 8, 1280)
1500
(155, 4, 8, 1280)
1501
(155, 4, 8, 1280)
1502
(155, 4, 8, 1280)
1503
(155, 4, 8, 1280)
1504
(155, 4, 8, 1280)
1505
(155, 4, 8, 1280)
1506
(155, 4, 8, 1280)
1507
(155, 4, 8, 1280)
1508
(155, 4, 8, 1280)
1509
(155, 4, 8, 1280)
1510
(155, 4, 8, 1280)
1511
(155, 4, 8, 1280)
1512
(155, 4, 8, 1280)
1513
(155, 4, 8, 1280)
1514
(155, 4, 8, 1280)
1515
(155, 4, 8, 1280)
1516
(155, 4, 8, 1280)
1517
(155, 4, 8,

...

(155, 4, 8, 1280)
4331
(155, 4, 8, 1280)
4332
(155, 4, 8, 1280)
4333
(155, 4, 8, 1280)
4334
(155, 4, 8, 1280)
4335
(155, 4, 8, 1280)
4336
(155, 4, 8, 1280)
4337
(155, 4, 8, 1280)
4338
(155, 4, 8, 1280)
4339
(155, 4, 8, 1280)
4340
(155, 4, 8, 1280)
4341
(155, 4, 8, 1280)
4342
(155, 4, 8, 1280)
4343
(155, 4, 8, 1280)
4344
(155, 4, 8, 1280)
4345
(155, 4, 8, 1280)
4346
(155, 4, 8, 1280)
4347
(155, 4, 8, 1280)
4348
(155, 4, 8, 1280)
4349
(155, 4, 8, 1280)
4350
(155, 4, 8, 1280)
4351
(155, 4, 8, 1280)
4352
(155, 4, 8, 1280)
4353
(155, 4, 8, 1280)
4354
(152, 4, 8, 1280)
4355
(155, 4, 8, 1280)
4356
(155, 4, 8, 1280)
4357
(155, 4, 8, 1280)
4358
(155, 4, 8, 1280)
4359
(155, 4, 8, 1280)
4360
(155, 4, 8, 1280)
4361
(155, 4, 8, 1280)
4362
(155, 4, 8, 1280)
4363
(155, 4, 8, 1280)
4364
(155, 4, 8, 1280)
4365
(155, 4, 8, 1280)
4366
(155, 4, 8, 1280)
4367
(155, 4, 8, 1280)
4368
(155, 4, 8, 1280)
4369
(155, 4, 8, 1280)
4370
(155, 4, 8, 1280)
4371
(155, 4, 8, 1280)
4372
(155, 4, 8, 1280)
4373
(155, 4, 8,

(155, 4, 8, 1280)
4688
(155, 4, 8, 1280)
4689
(155, 4, 8, 1280)
4690
(155, 4, 8, 1280)
4691
(155, 4, 8, 1280)
4692
(155, 4, 8, 1280)
4693
(155, 4, 8, 1280)
4694
(155, 4, 8, 1280)
4695
(155, 4, 8, 1280)
4696
(155, 4, 8, 1280)
4697
(155, 4, 8, 1280)
4698
(155, 4, 8, 1280)
4699
(155, 4, 8, 1280)
4700
(155, 4, 8, 1280)
4701
(155, 4, 8, 1280)
4702
(155, 4, 8, 1280)
4703
(155, 4, 8, 1280)
4704
(155, 4, 8, 1280)
4705
(155, 4, 8, 1280)
4706
(155, 4, 8, 1280)
4707
(155, 4, 8, 1280)
4708
(155, 4, 8, 1280)
4709
(155, 4, 8, 1280)
4710
(155, 4, 8, 1280)
4711
(155, 4, 8, 1280)
4712
(155, 4, 8, 1280)
4713
(155, 4, 8, 1280)
4714
(155, 4, 8, 1280)
4715
(155, 4, 8, 1280)
4716
(155, 4, 8, 1280)
4717
(155, 4, 8, 1280)
4718
(155, 4, 8, 1280)
4719
(155, 4, 8, 1280)
4720
(155, 4, 8, 1280)
4721
(155, 4, 8, 1280)
4722
(155, 4, 8, 1280)
4723
(155, 4, 8, 1280)
4724
(155, 4, 8, 1280)
4725
(155, 4, 8, 1280)
4726
(155, 4, 8, 1280)
4727
(155, 4, 8, 1280)
4728
(155, 4, 8, 1280)
4729
(155, 4, 8, 1280)
4730
(155, 4, 8,

(155, 4, 8, 1280)
5045
(155, 4, 8, 1280)
5046
(155, 4, 8, 1280)
5047
(155, 4, 8, 1280)
5048
(155, 4, 8, 1280)
5049
(155, 4, 8, 1280)
5050
(155, 4, 8, 1280)
5051
(155, 4, 8, 1280)
5052
(155, 4, 8, 1280)
5053
(155, 4, 8, 1280)
5054
(155, 4, 8, 1280)
5055
(155, 4, 8, 1280)
5056
(155, 4, 8, 1280)
5057
(155, 4, 8, 1280)
5058
(155, 4, 8, 1280)
5059
(155, 4, 8, 1280)
5060
(156, 4, 8, 1280)
5061
(155, 4, 8, 1280)
5062
(155, 4, 8, 1280)
5063
(155, 4, 8, 1280)
5064
(155, 4, 8, 1280)
5065
(155, 4, 8, 1280)
5066
(155, 4, 8, 1280)
5067
(155, 4, 8, 1280)
5068
(155, 4, 8, 1280)
5069
(155, 4, 8, 1280)
5070
(155, 4, 8, 1280)
5071
(156, 4, 8, 1280)
5072
(155, 4, 8, 1280)
5073
(155, 4, 8, 1280)
5074
(155, 4, 8, 1280)
5075
(155, 4, 8, 1280)
5076
(155, 4, 8, 1280)
5077
(155, 4, 8, 1280)
5078
(155, 4, 8, 1280)
5079
(155, 4, 8, 1280)
5080
(155, 4, 8, 1280)
5081
(155, 4, 8, 1280)
5082
(155, 4, 8, 1280)
5083
(155, 4, 8, 1280)
5084
(155, 4, 8, 1280)
5085
(155, 4, 8, 1280)
5086
(155, 4, 8, 1280)
5087
(155, 4, 8,

(155, 4, 8, 1280)
5402
(155, 4, 8, 1280)
5403
(155, 4, 8, 1280)
5404
(155, 4, 8, 1280)
5405
(155, 4, 8, 1280)
5406
(155, 4, 8, 1280)
5407
(155, 4, 8, 1280)
5408
(155, 4, 8, 1280)
5409
(118, 4, 8, 1280)
5410
(155, 4, 8, 1280)
5411
(155, 4, 8, 1280)
5412
(155, 4, 8, 1280)
5413
(155, 4, 8, 1280)
5414
(155, 4, 8, 1280)
5415
(155, 4, 8, 1280)
5416
(155, 4, 8, 1280)
5417
(155, 4, 8, 1280)
5418
(155, 4, 8, 1280)
5419
(156, 4, 8, 1280)
5420
(155, 4, 8, 1280)
5421
(155, 4, 8, 1280)
5422
(155, 4, 8, 1280)
5423
(155, 4, 8, 1280)
5424
(155, 4, 8, 1280)
5425
(155, 4, 8, 1280)
5426
(155, 4, 8, 1280)
5427
(155, 4, 8, 1280)
5428
(155, 4, 8, 1280)
5429
(155, 4, 8, 1280)
5430
(155, 4, 8, 1280)
5431
(155, 4, 8, 1280)
5432
(155, 4, 8, 1280)
5433
(155, 4, 8, 1280)
5434
(155, 4, 8, 1280)
5435
(155, 4, 8, 1280)
5436
(155, 4, 8, 1280)
5437
(155, 4, 8, 1280)
5438
(155, 4, 8, 1280)
5439
(155, 4, 8, 1280)
5440
(155, 4, 8, 1280)
5441
(155, 4, 8, 1280)
5442
(155, 4, 8, 1280)
5443
(155, 4, 8, 1280)
5444
(155, 4, 8,

(155, 4, 8, 1280)
5759
(155, 4, 8, 1280)
5760
(155, 4, 8, 1280)
5761
(155, 4, 8, 1280)
5762
(155, 4, 8, 1280)
5763
(155, 4, 8, 1280)
5764
(155, 4, 8, 1280)
5765
(155, 4, 8, 1280)
5766
(155, 4, 8, 1280)
5767
(155, 4, 8, 1280)
5768
(155, 4, 8, 1280)
5769
(155, 4, 8, 1280)
5770
(155, 4, 8, 1280)
5771
(155, 4, 8, 1280)
5772
(155, 4, 8, 1280)
5773
(155, 4, 8, 1280)
5774
(155, 4, 8, 1280)
5775
(155, 4, 8, 1280)
5776
(155, 4, 8, 1280)
5777
(155, 4, 8, 1280)
5778
(155, 4, 8, 1280)
5779
(155, 4, 8, 1280)
5780
(155, 4, 8, 1280)
5781
(155, 4, 8, 1280)
5782
(155, 4, 8, 1280)
5783
(86, 4, 8, 1280)
5784
(155, 4, 8, 1280)
5785
(155, 4, 8, 1280)
5786
(155, 4, 8, 1280)
5787
(155, 4, 8, 1280)
5788
(155, 4, 8, 1280)
5789
(155, 4, 8, 1280)
5790
(155, 4, 8, 1280)
5791
(155, 4, 8, 1280)
5792
(155, 4, 8, 1280)
5793
(155, 4, 8, 1280)
5794
(155, 4, 8, 1280)
5795
(155, 4, 8, 1280)
5796
(155, 4, 8, 1280)
5797
(155, 4, 8, 1280)
5798
(155, 4, 8, 1280)
5799
(155, 4, 8, 1280)
5800
(155, 4, 8, 1280)
5801
(155, 4, 8, 